In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/train.csv')
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.f = nn.Sequential( # B x 1 x 28 x 28
            nn.Conv2d(1, 32, 3, 1), # B x 32 x 26 x 26
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1), # B x 64 x 24 x 24
            nn.MaxPool2d(2), # B x 64 x 12 x 12
            nn.Dropout2d(0.25),
            nn.Flatten(1), # 9216
            nn.Linear(9216, 128), # 128
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(128, 10) # 10
        )

    def forward(self, x, y):
        # Cross entropy better for classification
        return - nn.functional.cross_entropy(self.f(x), y, reduction='none')

    def classify(self, x):
        # Computes the probability for a single image
        q = self.f(torch.as_tensor(x)[None, ...])[0]
        return nn.functional.softmax(q, dim=0)

In [33]:
class MNIST(Dataset):
    def __init__(self, x, y):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.x = torch.tensor(x.astype(np.float32)).to(device)
        self.y = torch.tensor(y.astype(np.float32)).to(device)
        
        self.n_samples = len(x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [42]:
def mnist_trained(x, y, batches=5, epochs=5):
    data = MNIST(x, y)
    loaded_data = DataLoader(dataset=data, batch_size=batches, shuffle=True)

    model = ImageClassifier()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in tqdm(range(epochs), desc="Training..."):
        for batch in loaded_data:
            x, y = batch
            
            # Zeros gradiant for training
            optimizer.zero_grad()
            
            # Calculates likelihood
            loglik = model(x, y)
            e = -torch.mean(loglik)
            
            # Updates parameters
            e.backward()
            optimizer.step()

In [43]:
x = df.iloc[:, 1:].to_numpy()
y = df.iloc[:, 1].to_numpy()

mnist_trained(x, y)

Training...:   0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [5, 784]